# Setup

In [ ]:
import os, pickle, glob
from bids import BIDSLayout

import numpy as np

import matplotlib.pyplot as plt

import nibabel as nib

import nilearn
from nilearn.input_data import NiftiMasker
from nilearn.image import get_data, math_img, index_img, load_img

import utils

from PIL import Image
import cv2 as cv

In [ ]:
print(' numpy: ', np.__version__, '\n',
      'nibabel: ', nib.__version__, '\n',
      'nilearn: ', nilearn.__version__, '\n')

In [ ]:
sub = "NB"
ses = "20201215"

raw_data_dir = os.path.abspath("/Users/smerdis/data/LGN/BIDS/NB_20201215/sub-NB/")
raw_layout = BIDSLayout(raw_data_dir, validate=False, derivatives=False)
derivs_dir = os.path.abspath('/Users/smerdis/data/LGN/BIDS/NB_20201215/derivatives/prf')
freesurfer_dir = os.path.abspath('/Users/smerdis/data/LGN/BIDS/NB_20201215/derivatives/freesurfer')

out_dir = os.path.abspath(f"{derivs_dir}/sub-{sub}")
preproc_layout = BIDSLayout(out_dir, validate=False)

anat_file = sorted([f for f in raw_layout.get(subject=sub, session=ses, suffix='T1w',
            extension=['nii.gz'], return_type='file')])[0]
print(f"anat file: {anat_file}")
raw_bolds_prf = sorted([f for f in raw_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii.gz'], return_type='file')])
mcf_bolds_prf = sorted([f for f in preproc_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii.gz'], return_type='file') if 'mcflirt' in f])

prf_mask_file = os.path.abspath(f"{out_dir}/sub-{sub}_ses-{ses}_mask.nii.gz")


In [ ]:
print('\n'.join(raw_bolds_prf))

In [ ]:
print('\n'.join(mcf_bolds_prf))

In [ ]:
prf_ref_vol_name = f"sub-{sub}_ses-{ses}_refvol.nii.gz"
prf_ref_vol_path = f"{out_dir}/{prf_ref_vol_name}"
print(prf_ref_vol_path)
!ls {prf_ref_vol_path}

# Preprocessing

## Make reference volume from first dicom

In [ ]:
first_dicom_prf = "/Users/smerdis/Dropbox/data/LGN/Nb_Prf_20201215/Silver_Arjun\ -\ 1/mb_bold_mb3_175iso_2sTR_5/IM-0005-0001.dcm"

In [ ]:
dcm2niix_cmd = f"dcm2niix -v 2 -s y -f {prf_ref_vol_name} -z y -o {out_dir} {first_dicom_prf}"
print(dcm2niix_cmd)

## Motion correct BOLD runs to this refvol

In [ ]:
def make_preproc_name(this_epi, desc, insert=True):
    epi_name = os.path.basename(this_epi)
    epi_stub = epi_name.split('.')[0]
    epi_stub_parts = epi_stub.split('_')
    if insert:
        epi_stub_parts.insert(-1, desc)
    else:
        epi_stub_parts[-2] = desc
    epi_stub_mcf = '_'.join(epi_stub_parts)
    return epi_stub_mcf

In [ ]:
for this_epi in raw_bolds_prf:
    epi_stub_mcf = make_preproc_name(this_epi, 'desc-mcflirt')
    if not os.path.isdir(os.path.join(out_dir, f"ses-{ses}", "func")):
        os.makedirs(os.path.join(out_dir, f"ses-{ses}", "func"))
    full_outpath = os.path.join(out_dir, f"ses-{ses}", "func", epi_stub_mcf)
    mcflirt_cmd = f"mcflirt -reffile {prf_ref_vol_path} -mats -plots -report -cost mutualinfo -smooth 16 -in {this_epi} -o {full_outpath}"
    print(mcflirt_cmd)
    #os.system(mcflirt_cmd)

## Make brainmask from first run

In [ ]:
!ls {prf_mask_file}

In [ ]:
prf_masker = NiftiMasker(detrend=False, standardize=False, mask_strategy="epi", t_r=2.0)
prf_masker.fit(mcf_bolds_prf[0])
prf_masker.mask_img_.to_filename(prf_mask_file)
prf_mask_img = prf_masker.mask_img_
print(prf_mask_file, prf_mask_img.shape)

## Read in, highpass filter, normalize to psc, and save

In [ ]:
m = NiftiMasker(mask_img=prf_mask_file, standardize='psc', detrend=True,
                high_pass=0.02, t_r=2, mask_strategy='epi', verbose=1)

In [ ]:
for epi in mcf_bolds_prf:
    preproc_fn = make_preproc_name(epi, 'desc-preproc', insert=False)
    full_outpath = os.path.join(out_dir, f"ses-{ses}", "func", preproc_fn)
    #preproc_data = m.fit_transform(epi)
    #preproc_img = m.inverse_transform(preproc_data)
    #print(full_outpath, preproc_img.shape)
    #preproc_img.to_filename(full_outpath)

In [ ]:
!ls /Users/smerdis/data/LGN/BIDS/NB_20201215/derivatives/prf/sub-NB/ses-20201215/func/sub-NB_ses-20201215_task-prf_run-01_desc-preproc_bold.nii

## Average

In [ ]:
i = load_img(preproc_bolds_prf[0])
print(i.affine)

In [ ]:
preproc_bolds_prf = sorted([f for f in preproc_layout.get(subject=sub, session=ses, task='prf', suffix='bold',
            extension=['nii'], return_type='file') if 'preproc' in f])

print('\n'.join(preproc_bolds_prf))
prf_input_img = utils.make_timeseries_for_prf(preproc_bolds_prf)

In [ ]:
prf_input_img

In [ ]:
nib.save(prf_input_img, 
         f"{out_dir}/sub-{sub}_ses-{ses}_preproc_mean_trim.nii")

# Make cortical mask

## Convert some freesurfer mgz files to nifti

In [ ]:
ribbon_mgz = f"{freesurfer_dir}/{sub}/mri/ribbon.mgz"
ribbon_nii = f"{out_dir}/ribbon.nii.gz"
brain_mgz = f"{freesurfer_dir}/{sub}/mri/brain.mgz"
brain_nii = f"{out_dir}/brain.nii.gz"

In [ ]:
print(f"cd {out_dir}")
print(f"freeview {brain_mgz} {ribbon_mgz}")

In [ ]:
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{brain_mgz} {brain_nii}"
print(mriconv_cmd)
os.system(mriconv_cmd)
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{freesurfer_dir}/{sub}/mri/wm.seg.mgz {out_dir}/wmseg.nii.gz"
print(mriconv_cmd)
os.system(mriconv_cmd)
mriconv_cmd = f"mri_convert --in_type mgz --out_type nii --out_orientation RAS \
{ribbon_mgz} {ribbon_nii}"
print(mriconv_cmd)
os.system(mriconv_cmd)

## Use FLIRT to register func ref vol to anat, save transform

In [ ]:
epi2struct_xfm = f"{out_dir}/epi2struct.mat"
struct2epi_xfm = f"{out_dir}/struct2epi.mat"

In [ ]:
flirt_cmd = f"flirt -in {prf_ref_vol_path} -ref {brain_nii} -wmseg \
{out_dir}/wmseg.nii.gz -dof 6 -omat {epi2struct_xfm} \
-out {out_dir}/mb3refvol2t1"
print(flirt_cmd)
os.system(flirt_cmd)

## Invert the transform and save it

In [ ]:
invxfm_cmd = f"convert_xfm -omat {struct2epi_xfm} -inverse {epi2struct_xfm}"
print(invxfm_cmd)
os.system(invxfm_cmd)

## Apply inverse transform to convert freesurfer outputs to functional space

In [ ]:
brain2func_cmd = f"flirt -ref {prf_ref_vol_path} -in {brain_nii} -out {out_dir}/brain_funcspace.nii.gz \
-init {struct2epi_xfm} -applyxfm"
print(brain2func_cmd)
os.system(brain2func_cmd)

## Try epi_reg from FSL to register epi to struct

In [ ]:
bet_t1_file = f"{out_dir}/sub-{sub}_ses-{ses}_desc-T1-bet.nii.gz"
print(f"bet {anat_file} {bet_t1_file}")

In [ ]:
epi2struct_epireg_xfm = f"{out_dir}/epi2struct_epireg.mat"
struct2epi_epireg_xfm = f"{out_dir}/struct2epi_epireg.mat"

In [ ]:
# uses FSL's FAST segmentation
epi_reg_cmd = f"epi_reg -v --epi={prf_ref_vol_path} --t1={anat_file} --t1brain={bet_t1_file} --out={epi2struct_epireg_xfm}"
print(epi_reg_cmd)

In [ ]:
# # with wmseg from freesurfer
# epi_reg_cmd2 = f"epi_reg -v --epi={prf_ref_vol_path} --t1={anat_file} --t1brain={bet_t1_file} --wmseg={out_dir}/wmseg.nii.gz --out=epi2struct_epireg"
# print(epi_reg_cmd2)

In [ ]:
# invert transform
invxfm_cmd = f"convert_xfm -omat {struct2epi_epireg_xfm} -inverse {epi2struct_epireg_xfm}"
print(invxfm_cmd)
os.system(invxfm_cmd)

In [ ]:
struct2epi_epireg_cmd = f"flirt -ref {prf_ref_vol_path} -in {brain_nii} -out {out_dir}/brain_funcspace_epireg.nii.gz \
-init {struct2epi_epireg_xfm} -applyxfm"
print(struct2epi_epireg_cmd)
os.system(struct2epi_epireg_cmd)

## Load ribbon and extract cortex only

In [ ]:
def make_func_parc_mask(ribbon_nii, parc_codes, xfm, out_fn):
    ribbon_img = load_img(ribbon_nii)
    ribbon_data = ribbon_img.get_data()
    cortex_mask = np.zeros_like(ribbon_data)
    print(np.count_nonzero(cortex_mask))
    for c in parc_codes:
        cortex_mask[ribbon_data==c] = 1
    print(np.count_nonzero(cortex_mask))
    cortex_mask_img = nib.Nifti1Image(cortex_mask, ribbon_img.affine)
    nib.save(cortex_mask_img, out_fn)
    cmd = f"flirt -ref {prf_ref_vol_path} -in {out_fn} -out {out_fn}_funcspace.nii.gz -init {xfm} -applyxfm"
    print(cmd)
    os.system(cmd)

In [ ]:
make_func_parc_mask(ribbon_nii, [3, 42], f"{out_dir}/cortex_mask")

In [ ]:
make_func_parc_mask(ribbon_nii, [3], f"{out_dir}/Lcortex_mask")
make_func_parc_mask(ribbon_nii, [42], f"{out_dir}/Rcortex_mask")

In [ ]:
ribbon2func_cmd = f"flirt -ref {prf_ref_vol_path} -in {ribbon_nii} -out {out_dir}/ribbon_funcspace.nii.gz \
-init {out_dir}/brain2func.mat -applyxfm"
print(ribbon2func_cmd)
os.system(ribbon2func_cmd)

In [ ]:
print(f"fsleyes {anat_file} {ribbon_nii}")

# Load Screenshots to make prfs better

In [ ]:
sshots_dir = f"{out_dir}/Screenshots"
imlist = [
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_001.png",
    f"{sshots_dir}/Sc_1NBTest_A_002.png",
    f"{sshots_dir}/Sc_1NBTest_A_003.png",
    f"{sshots_dir}/Sc_1NBTest_A_004.png",
    f"{sshots_dir}/Sc_1NBTest_A_005.png",
    f"{sshots_dir}/Sc_1NBTest_A_006.png",
    f"{sshots_dir}/Sc_1NBTest_A_007.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_008.png",
    f"{sshots_dir}/Sc_1NBTest_A_009.png",
    f"{sshots_dir}/Sc_1NBTest_A_010.png",
    f"{sshots_dir}/Sc_1NBTest_A_011.png",
    f"{sshots_dir}/Sc_1NBTest_A_012.png",
    f"{sshots_dir}/Sc_1NBTest_A_013.png",
    f"{sshots_dir}/Sc_1NBTest_A_014.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_015.png",
    f"{sshots_dir}/Sc_1NBTest_A_016.png",
    f"{sshots_dir}/Sc_1NBTest_A_017.png",
    f"{sshots_dir}/Sc_1NBTest_A_018.png",
    f"{sshots_dir}/Sc_1NBTest_A_019.png",
    f"{sshots_dir}/Sc_1NBTest_A_020.png",
    f"{sshots_dir}/Sc_1NBTest_A_021.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_022.png",
    f"{sshots_dir}/Sc_1NBTest_A_023.png",
    f"{sshots_dir}/Sc_1NBTest_A_024.png",
    f"{sshots_dir}/Sc_1NBTest_A_025.png",
    f"{sshots_dir}/Sc_1NBTest_A_026.png",
    f"{sshots_dir}/Sc_1NBTest_A_027.png",
    f"{sshots_dir}/Sc_1NBTest_A_028.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_029.png",
    f"{sshots_dir}/Sc_1NBTest_A_030.png",
    f"{sshots_dir}/Sc_1NBTest_A_031.png",
    f"{sshots_dir}/Sc_1NBTest_A_032.png",
    f"{sshots_dir}/Sc_1NBTest_A_033.png",
    f"{sshots_dir}/Sc_1NBTest_A_034.png",
    f"{sshots_dir}/Sc_1NBTest_A_035.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_036.png",
    f"{sshots_dir}/Sc_1NBTest_A_037.png",
    f"{sshots_dir}/Sc_1NBTest_A_038.png",
    f"{sshots_dir}/Sc_1NBTest_A_039.png",
    f"{sshots_dir}/Sc_1NBTest_A_040.png",
    f"{sshots_dir}/Sc_1NBTest_A_041.png",
    f"{sshots_dir}/Sc_1NBTest_A_042.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_043.png",
    f"{sshots_dir}/Sc_1NBTest_A_044.png",
    f"{sshots_dir}/Sc_1NBTest_A_045.png",
    f"{sshots_dir}/Sc_1NBTest_A_046.png",
    f"{sshots_dir}/Sc_1NBTest_A_047.png",
    f"{sshots_dir}/Sc_1NBTest_A_048.png",
    f"{sshots_dir}/Sc_1NBTest_A_049.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_050.png",
    f"{sshots_dir}/Sc_1NBTest_A_051.png",
    f"{sshots_dir}/Sc_1NBTest_A_052.png",
    f"{sshots_dir}/Sc_1NBTest_A_053.png",
    f"{sshots_dir}/Sc_1NBTest_A_054.png",
    f"{sshots_dir}/Sc_1NBTest_A_055.png",
    f"{sshots_dir}/Sc_1NBTest_A_056.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_001.png",
    f"{sshots_dir}/Sc_1NBTest_A_002.png",
    f"{sshots_dir}/Sc_1NBTest_A_003.png",
    f"{sshots_dir}/Sc_1NBTest_A_004.png",
    f"{sshots_dir}/Sc_1NBTest_A_005.png",
    f"{sshots_dir}/Sc_1NBTest_A_006.png",
    f"{sshots_dir}/Sc_1NBTest_A_007.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_008.png",
    f"{sshots_dir}/Sc_1NBTest_A_009.png",
    f"{sshots_dir}/Sc_1NBTest_A_010.png",
    f"{sshots_dir}/Sc_1NBTest_A_011.png",
    f"{sshots_dir}/Sc_1NBTest_A_012.png",
    f"{sshots_dir}/Sc_1NBTest_A_013.png",
    f"{sshots_dir}/Sc_1NBTest_A_014.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_015.png",
    f"{sshots_dir}/Sc_1NBTest_A_016.png",
    f"{sshots_dir}/Sc_1NBTest_A_017.png",
    f"{sshots_dir}/Sc_1NBTest_A_018.png",
    f"{sshots_dir}/Sc_1NBTest_A_019.png",
    f"{sshots_dir}/Sc_1NBTest_A_020.png",
    f"{sshots_dir}/Sc_1NBTest_A_021.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_022.png",
    f"{sshots_dir}/Sc_1NBTest_A_023.png",
    f"{sshots_dir}/Sc_1NBTest_A_024.png",
    f"{sshots_dir}/Sc_1NBTest_A_025.png",
    f"{sshots_dir}/Sc_1NBTest_A_026.png",
    f"{sshots_dir}/Sc_1NBTest_A_027.png",
    f"{sshots_dir}/Sc_1NBTest_A_028.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_029.png",
    f"{sshots_dir}/Sc_1NBTest_A_030.png",
    f"{sshots_dir}/Sc_1NBTest_A_031.png",
    f"{sshots_dir}/Sc_1NBTest_A_032.png",
    f"{sshots_dir}/Sc_1NBTest_A_033.png",
    f"{sshots_dir}/Sc_1NBTest_A_034.png",
    f"{sshots_dir}/Sc_1NBTest_A_035.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_036.png",
    f"{sshots_dir}/Sc_1NBTest_A_037.png",
    f"{sshots_dir}/Sc_1NBTest_A_038.png",
    f"{sshots_dir}/Sc_1NBTest_A_039.png",
    f"{sshots_dir}/Sc_1NBTest_A_040.png",
    f"{sshots_dir}/Sc_1NBTest_A_041.png",
    f"{sshots_dir}/Sc_1NBTest_A_042.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_043.png",
    f"{sshots_dir}/Sc_1NBTest_A_044.png",
    f"{sshots_dir}/Sc_1NBTest_A_045.png",
    f"{sshots_dir}/Sc_1NBTest_A_046.png",
    f"{sshots_dir}/Sc_1NBTest_A_047.png",
    f"{sshots_dir}/Sc_1NBTest_A_048.png",
    f"{sshots_dir}/Sc_1NBTest_A_049.png",
    f"{sshots_dir}/Blank.png",
    f"{sshots_dir}/Sc_1NBTest_A_050.png",
    f"{sshots_dir}/Sc_1NBTest_A_051.png",
    f"{sshots_dir}/Sc_1NBTest_A_052.png",
    f"{sshots_dir}/Sc_1NBTest_A_053.png",
    f"{sshots_dir}/Sc_1NBTest_A_054.png",
    f"{sshots_dir}/Sc_1NBTest_A_055.png",
    f"{sshots_dir}/Sc_1NBTest_A_056.png",
    f"{sshots_dir}/Blank.png",
]

In [ ]:
save_dir = sshots_dir

for i in range(len(imlist)):
    im = Image.open(imlist[i])
    if 'Blank' in imlist[i]:
        im.save(f"{save_dir}/screen{i+1:03d}.png",'PNG')
#     else:
#         im_mat = np.array(im)
#         im_mat[im_mat[:,:,0] != 128] = 0
#         im_mat[im_mat[:,:,1] != 128] = 0
#         im_mat[im_mat[:,:,2] != 128] = 0
#         plt.imshow(im_mat)
#         im_mat[im_mat[:,:,0] == 128] = -1
#         im_mat[im_mat[:,:,1] == 128] = -1
#         im_mat[im_mat[:,:,2] == 128] = -1
#         plt.imshow(im_mat)
#         img2ar = Image.fromarray(im_mat.astype('uint8'), 'RGB')
#         img2ar.save(f"{save_dir}/screen{i+1:03d}.png",'PNG')


#This will make the images but you should go over them individually to make sure that there are no stray pixels..
# else it will mess you pRF models
# The manual editing is required because of the complex visual stimuli.

In [ ]:
edited_sshots_dir = f"{sshots_dir}/edited"
for i, s in enumerate(sorted(glob.glob(f"{edited_sshots_dir}/screen*.png"))):
    print(i, s)
    im = Image.open(s)
    im.save(f"{sshots_dir}/screen{i+65:03d}.png")

In [ ]:
img_list = sorted(glob.glob(f"{sshots_dir}/screen*.png"))
blanks_to_pad = 138-len(img_list)
img_list.extend([img_list[-1]] * blanks_to_pad)

In [ ]:
len(img_list)

In [ ]:
#%%
biglist= [np.array([]) for _ in range(len(img_list))]
#%%

#This flips the images around to make it like how popeye uses it
    
for i in range(0,len(img_list)):
    im_read = cv.imread(img_list[i])
    im_rgb = cv.cvtColor(im_read, cv.COLOR_BGR2RGB)
    im_gr = cv.cvtColor(im_rgb, cv.COLOR_BGR2GRAY)
    # im_grd = np.divide(im_gr,255)
    print(img_list[i])
    ret,thresh = cv.threshold(im_gr,0,1,cv.THRESH_BINARY_INV)
    biglist[i] = thresh

bigstack = np.dstack((biglist))
pickle.dump(bigstack, open(f"{out_dir}/bigstack.pkl", 'wb'))


## Mask manipulation

In [ ]:
#m = nib.load(f"{out_dir}/cortex_mask_funcspace.nii.gz")
m = nib.load(f"{out_dir}/cortex_mask_funcspace.nii.gz")
m_chop = m.get_data().copy()
m_chop[:, 40:, :] = 0
chopped_mask_img = nib.Nifti1Image(m_chop, m.affine, m.header)
nib.save(chopped_mask_img, f"{out_dir}/sub-NB_ses-20201215_desc-cortexmask-back40_roi.nii.gz")

# Analyze the prfs

In [ ]:
# # import popeye
# # prf_file = f"{out_dir}/prf_20210202-231902.p"

# # with open(prf_file, 'rb') as file:
# #     prfs = pickle.load(file)

# # len(prfs)

# func_file = f"{out_dir}/sub-NB_ses-20201215_preproc_mean_trim.nii"
# func_img = nib.load(func_file)

# func_img.shape

# mask_factor = 0.5
# mask = nib.load(f"{out_dir}/cortex_mask_funcspace.nii.gz").get_data()#.astype(bool)
# mask = mask > mask_factor
# [xi,yi,zi] = np.where(mask)


# datadump = prfs

# #Make arrays to save as NIFTI file
# rsq = np.zeros(mask.shape)
# sig = np.zeros(mask.shape)
# rho = np.zeros(mask.shape)
# theta= np.zeros(mask.shape)
# x_x = np.zeros(mask.shape)
# y_y = np.zeros(mask.shape)

# for i in range(1,len(datadump),1):
#     rsq[datadump[i].voxel_index] = datadump[i].rsquared
#     sig[datadump[i].voxel_index] = datadump[i].sigma
#     rho[datadump[i].voxel_index] = datadump[i].rho
#     theta[datadump[i].voxel_index] = datadump[i].theta
#     # x_0[datadump[i].voxel_index] = datadump[i].x0
#     # y_0[datadump[i].voxel_index] = datadump[i].y0
#     x_x[datadump[i].voxel_index] = datadump[i].x
#     y_y[datadump[i].voxel_index] = datadump[i].y
# #%%
# rsq_img = nib.Nifti1Image(rsq,func_img.affine,header=func_img.header)
# sig_img = nib.Nifti1Image(sig,func_img.affine,header=func_img.header)
# rho_img = nib.Nifti1Image(rho,func_img.affine,header=func_img.header)
# theta_img = nib.Nifti1Image(theta,func_img.affine,header=func_img.header)
# # x_0_img = nb.Nifti1Image(x_0,func_img.affine)
# # y_0_img = nb.Nifti1Image(y_0,func_img.affine)
# x_x_img = nib.Nifti1Image(x_x,func_img.affine,header=func_img.header)
# y_y_img = nib.Nifti1Image(y_y,func_img.affine,header=func_img.header)
# mask_img = nib.Nifti1Image(mask,func_img.affine,header=func_img.header)
# imgs = (rsq_img, sig_img, rho_img, theta_img, x_x_img, y_y_img,mask_img)
# names = ('rsq','sig','rho','theta','xx','yy','mask')

# for (img, name) in zip(imgs, names):
#     print(img.shape, name)
#     nib.save(img, f"{out_dir}/sub-NB_ses-20201215_desc-prf{name}_space-func_map.nii.gz")

## Transform the prfs back to anatomical space for freeview

In [ ]:
prf_dir = f"{out_dir}/20210331-143142" # no screenshots, L cortex

In [ ]:
prf_outputs = glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*")
print("\n".join(prf_outputs))

In [ ]:
files_volspace = [x for x in prf_outputs if ('space-anat' not in x) and \
        (('rsq' in x) or ('rho' in x) or ('theta' in x)) \
                  and 'thresh' not in x
]
print('\n'.join(files_volspace))
print(f"fsleyes {anat_file} {prf_ref_vol_path} {' '.join(files_volspace)}")

In [ ]:
def threshold(what, by, at, out_dir, how='more'):
    """Threshold a given prf output (what) by another (by, usually rsq) at a specific value"""
    w = load_img(what)
    b = load_img(by)
    assert(w.shape == b.shape)
    if how=='more':
        mask = b.get_fdata() < at
    elif how=='less':
        mask = b.get_fdata() > at
    marray = np.ma.masked_array(w.get_fdata(), mask)
    out_data = np.ma.filled(marray, fill_value=0)
    print(f"Thresholding {what} by {by} at thresh {at:.2f}...\n", 
        mask.shape, out_data.shape, np.count_nonzero(mask), np.count_nonzero(out_data))
    out_img = nib.Nifti1Image(out_data, w.affine)
    parts_by = os.path.basename(by).split('_')
    desc_by = [p for p in parts_by if 'desc' in p][0].split('-')[1] # gets whatever is after desc-
    parts = os.path.basename(what).split('_')
    parts.insert(-1, f"thresh-{desc_by}-{at:.2f}")
    out_file = '_'.join(parts)
    print(desc_by, parts, out_file, sep='\n')
    nib.save(out_img, f"{out_dir}/{out_file}")

In [ ]:
rsq_file = [f for f in prf_outputs if 'desc-prfrsq' in f and 'thresh' not in f][0]

for output in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*"):
    if 'thresh-' not in output:
        print(output)
        #_ = threshold(output, rsq_file, .2, prf_dir)
        _ = threshold(output, rsq_file, .2, prf_dir)

In [ ]:
actual_mask = glob.glob(f"{prf_dir}/sub-{sub}*desc-prfmask*space-func*map*")[0]

files = [x for x in glob.glob(f"{prf_dir}/sub-{sub}*desc-prf[!mask]*map*") if ('space-anat' not in x)
                      and (('desc-prfrsq' in x) or ('desc-prfrho' in x) or ('desc-prftheta' in x))]
print('\n'.join(files))
print(f"fsleyes {anat_file} {prf_ref_vol_path} {actual_mask} {' '.join(files)}")

In [ ]:
print(f"fsleyes {anat_file} {out_dir}/brain.nii.gz {prf_ref_vol_path}")

In [ ]:
def prf_to_anat(brain_file, in_file, func2brain, out_dir):
    # put space-anat in there, replacing space-* if it exists
    parts = os.path.basename(in_file).split('_')
    parts = [p if 'space-' not in p else 'space-anat' for p in parts]
    if 'space-anat' not in parts:
        parts.insert(-1, 'space-anat')
    desc = [p for p in parts if 'desc' in p][0]
    try:
        thresh = [p for p in parts if 'thresh' in p][0]
    except IndexError as inst:
        thresh = 'nothresh'
    outname = f"{desc}-{thresh}"
    print(parts, outname)
    out_file = '_'.join(parts)
    cmd = f"flirt -ref {brain_file} -out {out_dir}/{out_file} -in {in_file} -init {func2brain} -applyxfm"
    print(cmd)
    os.system(cmd)
    for hemi in ("lh", "rh"):
        cmd2 = f"mri_vol2surf --src {out_dir}/{out_file} --o {out_dir}/{hemi}.{outname}.mgz --hemi {hemi} --regheader {sub} --projfrac 0.5"
        print(cmd2)
        os.system(cmd2)

In [ ]:
for output in prf_outputs:
    prf_to_anat(f"{out_dir}/brain.nii.gz", output, f"{out_dir}/func2brain.mat", prf_dir)

In [ ]:
def freeview_prfs(sub, hemi, prf_dir):
    """View prfs on surface in freeview with good colormap"""
    overlays = sorted(glob.glob(f"{prf_dir}/?h.desc-*.mgz"))
    disp_overlays = [x for x in overlays if ('rho' in x or 'theta' in x) and f"{hemi}." in x]
    print('\n'.join(disp_overlays))
    hemi_surf = f"$SUBJECTS_DIR/{sub}/surf/{hemi}.inflated"
    freeview_cmd = f"freeview -f {hemi_surf}"
    for x in disp_overlays:
        x_color = 'overlay_custom=0.01,255,0,0,1.57,125,255,0,3.14,0,255,255,4.71,125,0,255,6.28,255,0,0' if 'theta' in x else 'overlay_color=colorwheel'
        freeview_cmd = freeview_cmd + f":overlay={x}:{x_color}"
    print(freeview_cmd)

In [ ]:
utils.freeview_prfs(sub, 'lh', prf_dir)

In [ ]:
utils.freeview_prfs(sub, 'lh', prf_dir)